### Collect()
1. It is action operation 
2. Used to get data from the RDD and Dataframe from all nodes into the driver 
3. Usually will use after the filter(), group()
4. Using collect in larger dataset will result in the out of memory error


In [1]:
# creating spark session and df

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("learning").getOrCreate()


24/11/12 23:12:32 WARN Utils: Your hostname, padmanabhan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/11/12 23:12:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 23:12:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Data
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]
# Column names
columns = ["firstname","lastname","country","state"]

# Create DataFrame
df = spark.createDataFrame(data = data, schema = columns)

In [6]:
# collect - retrieve all elements in a dataframe from all nodes and return a row type obj
data =df.collect()

In [10]:
print(type(data))

for i in data:
    print(i)

<class 'list'>
Row(firstname='James', lastname='Smith', country='USA', state='CA')
Row(firstname='Michael', lastname='Rose', country='USA', state='NY')
Row(firstname='Robert', lastname='Williams', country='USA', state='CA')
Row(firstname='Maria', lastname='Jones', country='USA', state='FL')


In [12]:
# for specific columns

df.select("firstname").collect()

[Row(firstname='James'),
 Row(firstname='Michael'),
 Row(firstname='Robert'),
 Row(firstname='Maria')]

In [5]:
# 
from pyspark.sql.functions import col

data = [("padhu@gmail.com",)]
df = spark.createDataFrame(data,schema=["email"])


In [6]:
iterator = df.toLocalIterator()

In [9]:
iterator = df.toLocalIterator()
for i in iterator:
    valInd = i.email.index("@")
    print(i.email[valInd:])

@gmail.com


In [30]:
from pyspark.sql.functions import instr
Ind = df.select(instr(df.email,"@").alias("startInd")).collect()
Ind = Ind[0]["startInd"]

In [57]:
from pyspark.sql.functions import instr,length,substring,lit,expr
df2 = df.withColumn("startInd",instr(df.email,"@")).withColumn("endInd",length(df.email))
df2.show()

df3 = df.withColumn("answer",expr("substring(email,instr(email,'@'),length(email))"))
df3.show()

df4 = df2.withColumn("answer2",expr("substring(email,startInd,endInd)"))
df4.show()

df5 = df2.withColumn("answer3",substring(df2.email,col("startInd").cast('integer'),col("endInd").cast('integer')))
df5.show()

+---------------+--------+------+
|          email|startInd|endInd|
+---------------+--------+------+
|padhu@gmail.com|       6|    15|
+---------------+--------+------+

+---------------+----------+
|          email|    answer|
+---------------+----------+
|padhu@gmail.com|@gmail.com|
+---------------+----------+



+---------------+--------+------+----------+
|          email|startInd|endInd|   answer2|
+---------------+--------+------+----------+
|padhu@gmail.com|       6|    15|@gmail.com|
+---------------+--------+------+----------+



PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [48]:
df2.printSchema()

root
 |-- email: string (nullable = true)
 |-- startInd: integer (nullable = true)
 |-- endInd: integer (nullable = true)



In [38]:
from pyspark.sql.functions import substr,length

df.select(df.email.substr(Ind,df.email.len(df.email))).show()

TypeError: 'Column' object is not callable

In [ ]:
from pyspark.sql.functions import instr

df2 = df.withColumn("domain",df.email.substr(instr(df.email,"@"),11))

PySparkTypeError: [NOT_SAME_TYPE] Argument `startPos` and `length` should be the same type, got Column and int.

In [16]:
df2.show()

+---------------+------+
|          email|domain|
+---------------+------+
|padhu@gmail.com|  NULL|
+---------------+------+

